In [1]:
from bs4 import BeautifulSoup
import requests as req
import pandas as pd

In [2]:
def parse(link: str) -> pd.DataFrame:

    web = req.get(link)
    soup = BeautifulSoup(web.text, 'html.parser')

    df = pd.DataFrame([], columns=['name', 'price', 'old_price', 'sizes_available'])

    for i, el in enumerate(soup.findAll('div', class_='offer_item')):
        
        name = el.find('div', class_='name').a.text
        price = el.find('div', class_='price')

        if el.find('div', class_='sizes_available') is not None:
            sizes = [x.text for x in el.find('div', class_='sizes_available').find('div').findAll('a')]
            for j, size in enumerate(sizes):
                sizes[j] = ''.join([x for x in size if x.isalnum()])
        else:
            sizes = None

        if price.find('span', class_='old-price') is not None:

            spec_price = price.find('span', class_='spec-price').text
            old_price = price.find('span', class_='old-price').text

            df.loc[i] = [name, 
                        int(''.join([x for x in spec_price if x.isdigit()])), 
                        int(''.join([x for x in old_price if x.isdigit()])),
                        sizes]
        else:
            price_ = price.find('span').text
            df.loc[i] = [name, 
                        int(''.join([x for x in price_ if x.isdigit()])), 
                        None,
                        sizes]
            
    return df

In [3]:
link = 'https://21-shop.ru/catalog/muzhskoe/odezhda/bryuki/?PAGEN_1=3&more=1'

df = parse(link)
df

,name,price,old_price,sizes_available
0,Брюки SKILLS C&J Black,6900,None,"[XS, S, M, L, XL, 2XL]"
1,Брюки SKILLS C&J Grey,6900,None,"[S, M, L, XL, 2XL]"
2,Брюки SKILLS C&J Bottle Green,5520,6900,"[S, M, L, XL, 2XL]"
3,Джоггеры КРЕПОСТЬ Pocket Светлый Хаки,3179,3740,"[XS, XL, 2XL]"
4,Джоггеры КРЕПОСТЬ Pocket Серый,3179,3740,"[XS, S, M, L, XL, 2XL]"
5,Джоггеры КРЕПОСТЬ Pocket Черный,3179,3740,"[XS, S, L, 2XL]"
6,Джоггеры КРЕПОСТЬ Pocket Бежевый,3179,3740,"[XS, S, L, XL, 2XL]"
7,Брюки TRUESPIN FF-2205 Grey,2700,5400,[30]
8,Джоггеры КРЕПОСТЬ Basic Бежевый,3179,3740,"[XS, S, M, L, XL, 2XL]"
9,Джоггеры КРЕПОСТЬ Basic Черный,3179,3740,"[XS, S, M, L, XL, 2XL]"
